In [1]:
# 필요 라이브러리 정리
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.utils.class_weight import compute_class_weight

# 경고 무시
import warnings
warnings.filterwarnings("ignore")

# 한글 폰트 깨짐 없앰
plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus=False)

# 생활숲

In [2]:
# 통합데이터_생활숲타겟.xlsx을 생활숲에 저장
생활숲 = pd.read_excel('통합데이터_생활숲타겟.xlsx')
생활숲 = 생활숲.drop(columns= '동')

# X, y로 생활숲을 나눔
X = 생활숲.drop(columns=['target'])
y = 생활숲.loc[:, 'target']

# train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=24, stratify=y)

# StandardScaler 사용
life_scaler = StandardScaler()
X_train_scaled = life_scaler.fit_transform(X_train)
X_test_scaled = life_scaler.transform(X_test)

# 클래스 가중치 계산
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = {i : class_weights[i] for i in range(len(class_weights))}

# RandomForestClassifier 사용
life_model = RandomForestClassifier(class_weight=class_weights_dict, max_depth=3, n_estimators=100, random_state=24)
life_model.fit(X_train_scaled, y_train)
y_pred_life = life_model.predict(X_test_scaled)

# 평가 결과 출력
print(classification_report(y_test, y_pred_life))
print(confusion_matrix(y_test, y_pred_life))

              precision    recall  f1-score   support

           0       0.67      0.43      0.52        14
           1       0.76      0.89      0.82        28

    accuracy                           0.74        42
   macro avg       0.71      0.66      0.67        42
weighted avg       0.73      0.74      0.72        42

[[ 6  8]
 [ 3 25]]


In [3]:
## pkl파일로 저장
# joblib.dump(life_model, 'life_model.pkl')
# joblib.dump(life_scaler, 'life_scaler.pkl')

# 환경숲

- Fitting 3 folds for each of 24 candidates, totalling 72 fits
- 최적의 하이퍼파라미터: {'max_depth': 5, 'max_features': 'sqrt', 'n_estimators': 50}
- 최적의 교차 검증 점수 (정확도): 0.6099887766554433

In [4]:
# 통합데이터_환경숲타겟.xlsx 파일을 환경숲에 저장
환경숲 = pd.read_excel('통합데이터_환경숲타겟.xlsx')
환경숲 = 환경숲.drop('동', axis=1)

# 환경숲의 지목데이터의 중요도를 확인하여 상위 5개 데이터만 1로 설정함
# 우리나라 토지 지목 리스트를 columns_to_convert에 저장
columns_to_convert = [
    "전", "답", "과수원", "목장용지", "임야", "염전", "대", "공장용지", "학교용지", "주차장",
    "주유소용지", "창고용지", "도로", "철도용지", "하천", "제방", "구거", "유지", "양어장", "수도용지",
    "공원", "체육용지", "유원지", "종교용지", "사적지", "묘지", "잡종지", "광천지"
                    ]

# 각 행에 대해 상위 5개 값만 1로 설정하고 나머지는 0으로 설정
def top_n_to_one(row, n):
    row_copy = row.copy()
    top_n_indices = row[columns_to_convert].nlargest(n).index
    row_copy[columns_to_convert] = 0  # Initialize all to 0
    row_copy[top_n_indices] = 1  # Set top N to 1
    return row_copy

환경숲 = 환경숲.apply(lambda row: top_n_to_one(row, 5), axis=1)

X = 환경숲.drop('target', axis=1)
y = 환경숲['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    stratify=y,   # y클래스 비율에 맞게 분리
                                                    random_state=42)

# MinMaxScaler 사용
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# RandomForestClassifier 사용
rfc_lst = RandomForestClassifier(random_state=42, 
                                 max_depth=5, 
                                 n_estimators=50, 
                                 max_features='sqrt')
rfc_lst.fit(X_train_scaled, y_train)

y_pred_lst = rfc_lst.predict(X_test_scaled)

print('환경숲 정확도: ', accuracy_score(y_test, y_pred_lst))
print('Confusion Matrics: \n', confusion_matrix(y_test, y_pred_lst))

# confusion_matrix 시각화
plt.figure(figsize=(5, 3))
sns.heatmap(confusion_matrix(y_test, y_pred_lst), annot=True, cmap='Blues', cbar=False, annot_kws={'size': 15})
plt.title('Confusion Matrix - Test Data')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

# classification_report
print("\nClassification Report")
print(classification_report(y_test, y_pred_lst))

NameError: name 'accuracy_score' is not defined

In [ ]:
## pkl파일 저장
# joblib.dump(scaler, 'env_scaler.pkl')
# joblib.dump(rfc_lst, 'env_model.pkl')